In [1]:
## import libaries
import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm

In [2]:
## load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# function to read image
def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (139,139))
    return img

In [4]:
## set path for images
TRAIN_PATH = 'train_img/'
TEST_PATH = 'test_img/'

In [5]:
# load data
train_img, test_img = [],[]
for img_path in tqdm(train['image_id'].values):
    train_img.append(read_img(TRAIN_PATH + img_path + '.png'))

for img_path in tqdm(test['image_id'].values):
    test_img.append(read_img(TEST_PATH + img_path + '.png'))

100%|██████████| 1732/1732 [00:04<00:00, 388.95it/s]


In [6]:
# normalize images
x_train = np.array(train_img, np.float32) / 255.
x_test = np.array(test_img, np.float32) / 255.

In [7]:
# target variable - encoding numeric value
label_list = train['label'].tolist()
Y_train = {k:v+1 for v,k in enumerate(set(label_list))}
y_train = [Y_train[k] for k in label_list]   
y_train = np.array(y_train)

In [8]:
from keras import applications
from keras.models import Model
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.metrics import categorical_accuracy
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [9]:
y_train = to_categorical(y_train)

In [10]:
#Transfer learning with Inception V3 
base_model = applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(139, 139, 3))

In [11]:
## set model architechture 
add_model = Sequential()
add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
add_model.add(Dense(256, activation='relu'))
add_model.add(Dense(y_train.shape[1], activation='softmax'))

model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=1e-4),
              metrics=['accuracy'])

model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 139, 139, 3)   0                                            
____________________________________________________________________________________________________
conv2d_1 (Conv2D)                (None, 69, 69, 32)    864         input_1[0][0]                    
____________________________________________________________________________________________________
batch_normalization_1 (BatchNorm (None, 69, 69, 32)    96          conv2d_1[0][0]                   
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 69, 69, 32)    0           batch_normalization_1[0][0]      
___________________________________________________________________________________________

In [12]:
batch_size = 28 # tune it
epochs = 20 # increase it

train_datagen = ImageDataGenerator(
        rotation_range=30, 
        width_shift_range=0.1,
        height_shift_range=0.1, 
        horizontal_flip=True)
train_datagen.fit(x_train)

In [13]:
history = model.fit_generator(
    train_datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=x_train.shape[0] // batch_size,
    epochs=epochs,
    callbacks=[ModelCheckpoint('Inception-transferlearning.model', monitor='val_acc', save_best_only=True)]
)

Epoch 1/20
114/114 [==============================] - 34s - loss: 2.8388 - acc: 0.2046    

/home/daksh/.conda/envs/ai/lib/python3.6/site-packages/keras/callbacks.py:405: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)



Epoch 2/20
114/114 [==============================] - 29s - loss: 1.8755 - acc: 0.4501    
Epoch 3/20
114/114 [==============================] - 29s - loss: 1.3409 - acc: 0.5923    
Epoch 4/20
114/114 [==============================] - 29s - loss: 0.9291 - acc: 0.7145    
Epoch 5/20
114/114 [==============================] - 28s - loss: 0.7086 - acc: 0.7782    
Epoch 6/20
114/114 [==============================] - 28s - loss: 0.5227 - acc: 0.8399    
Epoch 7/20
114/114 [==============================] - 28s - loss: 0.4133 - acc: 0.8691    
Epoch 8/20
114/114 [==============================] - 28s - loss: 0.3392 - acc: 0.8897    
Epoch 9/20
114/114 [==============================] - 29s - loss: 0.2514 - acc: 0.9192    
Epoch 10/20
114/114 [==============================] - 29s - loss: 0.2368 - acc: 0.9275    
Epoch 11/20
114/114 [==============================] - 29s - loss: 0.1897 - acc: 0.9439    
Epoch 12/20
114/114 [==============================] - 29s - loss: 0.1805 - acc: 0.9392

In [14]:
## predict test data
predictions = model.predict(x_test)


In [15]:
# get labels
predictions = np.argmax(predictions, axis=1)
rev_y = {v:k for k,v in Y_train.items()}
pred_labels = [rev_y[k] for k in predictions]

In [16]:
## make submission
sub = pd.DataFrame({'image_id':test.image_id, 'label':pred_labels})
sub.to_csv('submitinception.csv', index=False) ## ~0.59